## BBC - Project
### Data Exploration

In [1]:
DATA_PATH = './data/'

In [2]:
import numpy as np
from Bio import Geo

def load_geo(myfile):
    handle = open(myfile)
    records = Geo.parse(handle)
    return records
    
records = load_geo(DATA_PATH + 'GSE21510_family.soft')

The following cell should be executed multiple times to run through the data

In [3]:
nr = next(records)
print(nr)
print('\n---------------------- ENTITY_ATTRIBUTES:------------------------------\n')
print(nr.entity_attributes)

GEO Type: DATABASE
GEO Id: GeoMiame
Database_email: geo@ncbi.nlm.nih.gov

Database_institute: NCBI NLM NIH

Database_name: Gene Expression Omnibus (GEO)

Database_web_link: http://www.ncbi.nlm.nih.gov/geo

Column Header Definitions


---------------------- ENTITY_ATTRIBUTES:------------------------------

{'Database_name': 'Gene Expression Omnibus (GEO)', 'Database_institute': 'NCBI NLM NIH', 'Database_web_link': 'http://www.ncbi.nlm.nih.gov/geo', 'Database_email': 'geo@ncbi.nlm.nih.gov'}


In [4]:
# re-loading the data
records = load_geo(DATA_PATH + 'GSE21510_family.soft')
series_sample_id = []
sample_titles = []
genes = []
nb_cols = 0
nb_rows = 0
data = []
for r in records:
    rea = r.entity_attributes
    if 'Series_geo_accession' in rea:
        if rea['Series_geo_accession'] == 'GSE21510':
            series_sample_id = rea['Series_sample_id']
            nb_cols = len(series_sample_id)
    if 'Sample_title' in rea:
        sample_titles.append(rea['Sample_title'])
        if 'sample_table_begin' in rea:
            nb_rows = rea['Sample_data_row_count'] 
            data.append(r.table_rows)
                    

data will become a 2d numpy array of shape (54676, 149) == (NB_GENES+1, NB_SAMPLES+1).  
First row will contain the sample ids, first column will contain the ID_REFs of the genes.

In [5]:
all_data = np.ndarray((int(nb_rows)+1, int(nb_cols)+1), dtype=object)
# labels
all_data[0, 0] = 'ID_REF'
all_data[0, 1:] = np.array(series_sample_id)

for i, d in enumerate(data):
    values = np.array(d[1:])
    if (i == 0):
        all_data[1:, 0] = values[:, 0]
    all_data[1:, i+1] = values[:, 1]
data = np.array(all_data)

In [6]:
data.shape

(54676, 149)

In [13]:
print(series_sample_id)
print('')
print(sample_titles)
print('')
print(len(data), len(data[0]))
print(data[0:10, 0])

['GSM537330', 'GSM537331', 'GSM537332', 'GSM537333', 'GSM537334', 'GSM537335', 'GSM537336', 'GSM537337', 'GSM537338', 'GSM537339', 'GSM537340', 'GSM537341', 'GSM537342', 'GSM537343', 'GSM537344', 'GSM537345', 'GSM537346', 'GSM537347', 'GSM537348', 'GSM537349', 'GSM537350', 'GSM537351', 'GSM537352', 'GSM537353', 'GSM537354', 'GSM537355', 'GSM537356', 'GSM537357', 'GSM537358', 'GSM537359', 'GSM537360', 'GSM537361', 'GSM537362', 'GSM537363', 'GSM537364', 'GSM537365', 'GSM537366', 'GSM537367', 'GSM537368', 'GSM537369', 'GSM537370', 'GSM537371', 'GSM537372', 'GSM537373', 'GSM537374', 'GSM537375', 'GSM537376', 'GSM537377', 'GSM537378', 'GSM537379', 'GSM537380', 'GSM537381', 'GSM537382', 'GSM537383', 'GSM537384', 'GSM537385', 'GSM537386', 'GSM537387', 'GSM537388', 'GSM537389', 'GSM537390', 'GSM537391', 'GSM537392', 'GSM537393', 'GSM537394', 'GSM537395', 'GSM537396', 'GSM537397', 'GSM537398', 'GSM537399', 'GSM537400', 'GSM537401', 'GSM537402', 'GSM537403', 'GSM537404', 'GSM537405', 'GSM537406'

In [8]:
import re

samples = {}
patient_names = []
for i_t, title in enumerate(sample_titles):
    split = title.split(',')
    if(re.search('cancer', title)):
        samples[series_sample_id[i_t]] = 'cancer'
    else:
        samples[series_sample_id[i_t]] = 'normal'
        
    patient_names.append(split[0])
        
samples

{'GSM537330': 'cancer',
 'GSM537331': 'cancer',
 'GSM537332': 'cancer',
 'GSM537333': 'cancer',
 'GSM537334': 'cancer',
 'GSM537335': 'cancer',
 'GSM537336': 'cancer',
 'GSM537337': 'cancer',
 'GSM537338': 'cancer',
 'GSM537339': 'cancer',
 'GSM537340': 'cancer',
 'GSM537341': 'cancer',
 'GSM537342': 'cancer',
 'GSM537343': 'cancer',
 'GSM537344': 'cancer',
 'GSM537345': 'cancer',
 'GSM537346': 'cancer',
 'GSM537347': 'cancer',
 'GSM537348': 'cancer',
 'GSM537349': 'cancer',
 'GSM537350': 'cancer',
 'GSM537351': 'cancer',
 'GSM537352': 'cancer',
 'GSM537353': 'cancer',
 'GSM537354': 'cancer',
 'GSM537355': 'cancer',
 'GSM537356': 'cancer',
 'GSM537357': 'cancer',
 'GSM537358': 'cancer',
 'GSM537359': 'cancer',
 'GSM537360': 'cancer',
 'GSM537361': 'cancer',
 'GSM537362': 'cancer',
 'GSM537363': 'cancer',
 'GSM537364': 'cancer',
 'GSM537365': 'cancer',
 'GSM537366': 'cancer',
 'GSM537367': 'cancer',
 'GSM537368': 'cancer',
 'GSM537369': 'cancer',
 'GSM537370': 'cancer',
 'GSM537371': 'c

In [9]:
print(sample_titles[104])

patient 107, normal, homogenized [OPG study]


### Prepare the data structure to run the SAM tests

Creating a dummy dataset in the Excel file for the SAM analysis

In [29]:
import pandas as pd

In [58]:
ar = np.ndarray((7, 5), dtype=object)
# ar[0][0] = 'N/A'
# ar[0][1] = 'N/A'
ar[0, 2:] = np.array([1, 2, 1], dtype='int32')
ar[1:, 0] = np.array(['g1', 'g2', 'g3', 'g4', 'g5', 'g6'])
ar[1:, 1] = np.array(['1001_at', '1002_at', '1003_at', '1004_at', '1005_at', '1006_at'])
ar[1:, 2:] = np.zeros((6, 3))
ar

array([[None, None, 1, 2, 1],
       ['g1', '1001_at', 0.0, 0.0, 0.0],
       ['g2', '1002_at', 0.0, 0.0, 0.0],
       ['g3', '1003_at', 0.0, 0.0, 0.0],
       ['g4', '1004_at', 0.0, 0.0, 0.0],
       ['g5', '1005_at', 0.0, 0.0, 0.0],
       ['g6', '1006_at', 0.0, 0.0, 0.0]], dtype=object)

In [59]:
ar_frame = pd.DataFrame(ar)

In [60]:
ar_frame

,0,1,2,3,4
0,None,None,1,2,1
1,g1,1001_at,0,0,0
2,g2,1002_at,0,0,0
3,g3,1003_at,0,0,0
4,g4,1004_at,0,0,0
5,g5,1005_at,0,0,0
6,g6,1006_at,0,0,0


In [61]:
writer = pd.ExcelWriter('ar.xlsx')
ar_frame.to_excel(writer, sheet_name='ar', index=False, header=False)
writer.save()